# Preamble

In [2]:
HomeDir = '../'
DataDir = HomeDir+'data/'
ListDir = HomeDir+'lists/'
ListTauDir = HomeDir+'lists/coarse_tau/'

import healpy as hp
import numpy as np
import math
import copy
from tqdm import tqdm, tqdm_notebook
from time import time as tictoc
import pandas as pd
from scipy.interpolate import griddata
from scipy import interpolate
import sys
import os
from joblib import Parallel, delayed
import multiprocessing

from MyUnits import *

# Functions

## Angular separation

In [2]:
def angular_sep(ra1, dec1, ra2, dec2):
    """Computes 2d angular separations vector for stars close to each other"""
    return np.array([(ra1-ra2)*np.cos((dec1+dec2)/2), (dec1-dec2)]).T

## Equatorial to ecliptic coordinate transformation

From https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html and section 1.5.3 of https://www.cosmos.esa.int/documents/532822/552851/vol1_all.pdf

In [4]:
rot_matrix = np.array([[1, 0, 0], [0, 0.9174821334228558, 0.39777699135300065], [0, -0.39777699135300065, 0.9174821334228558]])
ra_offset = 0.05542*arcsec
  
def fn_eq_to_ecl_array(ra, dec):
    """Function to convert the equatoria coordinates (ra, dec) to ecliptic longitude and latitude according to the Gaia reference frame"""
    """Works only if ra and dec are numpy arrays. Takes angle in deg and returns in deg"""
    
    ra_s, dec_s = ra*degree + ra_offset, dec*degree
    x_vec_eq = np.array([np.cos(dec_s)*np.cos(ra_s), np.cos(dec_s)*np.sin(ra_s), np.sin(dec_s)])
    x_vec_ecl = (rot_matrix @ x_vec_eq).T
    
    ecl_lon, ecl_lat = (np.arctan2(x_vec_ecl[:, 1], x_vec_ecl[:, 0])), np.arctan2(x_vec_ecl[:, 2], np.sqrt(x_vec_ecl[:, 0]**2 + x_vec_ecl[:, 1]**2))
    ecl_lon = ecl_lon + 2*np.pi*np.heaviside(-ecl_lon, 0) ### shift the interval from [-pi, pi] to [0, 2*pi]

    return ecl_lon/degree, ecl_lat/degree

## Matched filter

In [ ]:
# Uploading lists for the G_0 function and it's derivative. G_0 is the enclosed lens mass within a cylinder oriented along the line of sight. 
# See Eq.(3.10)=(3.11) of 1804.01991 or Eq.(2) of 2002.01938
# For the NFW truncated lens profile given by Eq.(3) of 2002.01938 the enclosed mass cannot be computed analytically. We use an interpolation function.

logxG0_list = np.loadtxt(ListDir+'G0NFWtrunc.txt');  logxG0_prime_list = np.loadtxt(ListDir+'G0pNFWtrunc.txt');  #logxG0_second_list = np.loadtxt(ListDir+'G0ppNFWtrunc.txt');
logG0_fnc = interpolate.interp1d(logxG0_list[:, 0], logxG0_list[:, 1], kind='cubic', bounds_error=False, fill_value=(logxG0_list[0, 1], logxG0_list[-1, 1]))
logG0_p_fnc = interpolate.interp1d(logxG0_prime_list[:, 0], logxG0_prime_list[:, 1], kind='cubic', bounds_error=False, fill_value=(logxG0_prime_list[0, 1], logxG0_prime_list[-1, 1]))

"Returns the lens enclosed mass within the distance x = beta/beta_l"
def G0_fnc(x): return np.power(10, logG0_fnc(np.log10(x+1E-20)))
def G0_p_fnc(x): return np.power(10, logG0_p_fnc(np.log10(x+1E-20)))

def dipole_mf(b_l, b_vec):
    """Returns the pm dipole-like profile"""
    b_norm = np.sqrt(b_vec[:, 0]**2 + b_vec[:, 1]**2)
    b_hat = np.array([b_vec[:,0]/(b_norm+1E-20), b_vec[:,1]/(b_norm+1E-20)]).T; x = b_norm/b_l
    G0_over_xsq = G0_fnc(x)/(x**2+1E-20); G0p_over_x = G0_p_fnc(x)/(x+1E-20)

    remove_inf = np.heaviside(b_norm, 0) # set to zero values corresponding to b_vec = [0, 0], remove infinity at the origin
    
    dipole_ra = np.array([(G0_over_xsq*(2*b_hat[:, 0]*b_hat[:, 0] - 1) - G0p_over_x*b_hat[:,0]*b_hat[:,0])*remove_inf, 
                          (G0_over_xsq*(2*b_hat[:, 1]*b_hat[:, 0]) - G0p_over_x*b_hat[:, 1]*b_hat[:, 0])*remove_inf]).T
    dipole_dec = np.array([(G0_over_xsq*(2*b_hat[:, 0]*b_hat[:, 1]) - G0p_over_x*b_hat[:, 1]*b_hat[:, 0])*remove_inf, 
                           (G0_over_xsq*(2*b_hat[:, 1]*b_hat[:, 1] - 1) - G0p_over_x*b_hat[:, 1]*b_hat[:, 1])*remove_inf]).T
    isotropic_dipole_magn = (G0_over_xsq**2 + 0.5*(G0p_over_x**2-2*G0_over_xsq*G0p_over_x))*remove_inf # for the normalization; see Eq. (13) of 2002.01938
    
    ### compute the monopole to check the background
    monopole = np.array([(G0_over_xsq*b_hat[:, 0])*remove_inf, (G0_over_xsq*b_hat[:, 1])*remove_inf]).T
    monopole_magn = G0_over_xsq**2*remove_inf
        
    return dipole_ra, dipole_dec, isotropic_dipole_magn, monopole, monopole_magn

In [16]:
def parallax_mf(b_l, b_vec, sinb):
    """Returns the parallax profile"""
    b_norm = np.sqrt(b_vec[:, 0]**2 + b_vec[:, 1]**2)
    b_hat = np.array([b_vec[:,0]/(b_norm+1E-20), b_vec[:,1]/(b_norm+1E-20)]).T; x = b_norm/b_l
    G0_over_xsq = G0_fnc(x)/(x**2+1E-20); G0p_over_x = G0_p_fnc(x)/(x+1E-20)
       
    return -(G0_over_xsq*(2*b_hat[:, 1]**2-1)*(1-sinb**2) + G0p_over_x*(1 - (1-sinb**2)*b_hat[:, 1]**2))/(1+sinb**2)   

## Template

In [19]:
def fn_template_scan(nside, coarse_scan_pix):
    """Compute the template at the locations given by coarse_scan_pix"""
    """Includes pm and parallax templates"""
    
    coarse_scan_coord = hp.pix2ang(nside, coarse_scan_pix, nest=True, lonlat=True) # coordinates of the template locations

    ### Cartesian vectors for each pixel in coarse_scan_pix
    vec_pix_x, vec_pix_y, vec_pix_z = hp.pix2vec(nside, coarse_scan_pix, nest=True) 
    vec_array = np.array([vec_pix_x, vec_pix_y, vec_pix_z]).T
    
    n_loc = len(coarse_scan_pix)
    tau_mu_ra, tau_mu_dec, tau_mu_norm = np.zeros(n_loc), np.zeros(n_loc), np.zeros(n_loc)
    tau_mu_mon, tau_mu_mon_norm = np.zeros(n_loc), np.zeros(n_loc)
    tau_par, tau_par_norm = np.zeros(n_loc), np.zeros(n_loc)
    
    for i in range(n_loc):
        nb_pix_i = hp.query_disc(nside, vec_array[i], n_betat*beta_t, inclusive=True, nest=True) # disc around the template position 

        stars_in = ((q_pix >= nb_pix_i[0]) & (q_pix <= nb_pix_i[-1])) # first reduce the total number of stars   
        nb_stars = np.isin(q_pix[stars_in], nb_pix_i, assume_unique=False, invert=False) # keep only stars within the neighboring pixels 

        ### Pm template
        beta_it = angular_sep(coarse_scan_coord[0][i]*degree, coarse_scan_coord[1][i]*degree, data_ra[stars_in][nb_stars]*degree, data_dec[stars_in][nb_stars]*degree)
        mu_ra, mu_dec, mu_sq, mu_mon, mu_mon_sq = dipole_mf(beta_t, beta_it)

        tau_mu_norm[i] = np.sqrt(sum(mu_sq/pm_w_sq[stars_in][nb_stars])) ## normalization
        tau_mu_ra[i] = sum((mu_ra[:, 0]*weighted_pmra[stars_in][nb_stars] + mu_ra[:, 1]*weighted_pmdec[stars_in][nb_stars]))
        tau_mu_dec[i] = sum((mu_dec[:, 0]*weighted_pmra[stars_in][nb_stars] + mu_dec[:, 1]*weighted_pmdec[stars_in][nb_stars]))
        
        tau_mu_mon_norm[i] = np.sqrt(sum(mu_mon_sq/pm_w_sq[stars_in][nb_stars])) ## normalization
        tau_mu_mon[i] = sum((mu_mon[:, 0]*weighted_pmra[stars_in][nb_stars] + mu_mon[:, 1]*weighted_pmdec[stars_in][nb_stars]))
        
        ### Parallax template
        beta_it_ecl = angular_sep(coarse_scan_coord_ecl[0][i]*degree, coarse_scan_coord_ecl[1][i]*degree, data_ecl_lon[stars_in][nb_stars]*degree, data_ecl_lat[stars_in][nb_stars]*degree)
        par_t = parallax_mf(beta_t, beta_it_ecl, np.sin(coarse_scan_coord_ecl[1][i]*degree)) 
        
        tau_par_norm[i] = np.sqrt(sum(par_t**2/par_w_sq[stars_in][nb_stars])) ## normalization
        tau_par[i] =  sum(par_t*weighted_par[stars_in][nb_stars])
        
    np.save(ListTauDir+'tau_b'+beta_t_deg+'_'+str(i_step), np.array([coarse_scan_coord[0], coarse_scan_coord[1], tau_mu_ra, tau_mu_dec, tau_mu_norm, tau_mu_mon, tau_mu_mon_norm, tau_par, tau_par_norm]).T)
    
    return None

# Run

## Read in and prepare data

In [3]:
### Define disc on the sky where the analysis is done
disc_radius = 5*degree; disc_center = np.array([81.28,-69.78]) 

In [7]:
data_np = np.load(DataDir+'LMC_disc_5_final.npy') ### Much faster than reading in a csv file
### Columns of the numpy array are are: 
### [data['ra'].to_numpy(), data['dec'].to_numpy(), data['pm_eff_error'].to_numpy()**2, data['pmra_sub'].to_numpy()/data['pm_eff_error'].to_numpy()**2, data['pmdec_sub'].to_numpy()/data['pm_eff_error'].to_numpy()**2, 
###  data['parallax_eff_error'].to_numpy()**2, data['parallax_sub'].to_numpy()/data['parallax_eff_error'].to_numpy()**2]
[data_ra, data_dec, data_ecl_lon, data_ecl_lat, pm_w_sq, weighted_pmra, weighted_pmdec, par_w_sq, weighted_par] = data_np.T
del data_np

In [ ]:
beta_kernel_sub_0 = 0.1*degree; # gaussian kernel for background subtraction 
beta_kernel_sub = 0.06*degree; # 0.1*degree; gaussian kernel for background subtraction 
n_iter_sub = 3; n_betat=3.5;
beta_t_deg = sys.argv[1] #str(20)
beta_t = float(beta_t_deg)/10000*degree
n_steps = int(sys.argv[2]) #40
i_step = int(sys.argv[3]) #0

In [4]:
#beta_kernel_sub_0 = 0.1*degree; # gaussian kernel for background subtraction 
#beta_kernel_sub = 0.06*degree; # 0.1*degree; gaussian kernel for background subtraction 
#n_iter_sub = 3; n_betat=3.5;
#beta_t_deg = str(90)
#beta_t = float(beta_t_deg)/10000*degree
#n_steps = 40
#i_step = 0

## Template scan

Split the patch of the sky into subpatches to run the template scan in parallel

In [5]:
### Coarse pixelation of size approx. beta_t. Determine how many template scan location there are
n = math.ceil(math.log(np.sqrt(np.pi/3)/beta_t, 2)); nside = 2**n; 
vec = hp.pix2vec(nside, hp.ang2pix(nside, disc_center[0], disc_center[1], nest=True, lonlat=True), nest=True)
disc_pix_coarse = hp.query_disc(nside, vec, disc_radius - beta_kernel_sub_0 - (n_iter_sub+1)*beta_kernel_sub - n_betat*beta_t, nest=True, inclusive=False) # pixels on the sky within a disc without the edge
n_locations = len(disc_pix_coarse)
step = math.ceil(n_locations/n_steps)

print('Template scan for beta_t = '+str(beta_t/degree)+' deg.')
print('Number of template locations: '+str(n_locations)+'. Dividing in '+str(n_steps)+' steps. Number of locations per step = '+str(step)+'.')
sys.stdout.flush()

Template scan for beta_t = 0.009 deg.
Number of template locations: 1279310. Dividing in 40 steps. Number of locations per step = 31983.


In [10]:
coarse_scan_coord = hp.pix2ang(nside, disc_pix_coarse, nest=True, lonlat=True) # to be converted into pixels at the fine pixelation scale below
coarse_scan_coord_ecl = fn_eq_to_ecl_array(coarse_scan_coord[0], coarse_scan_coord[1]) # convert into ecliptic coordinates for the parallax template
del vec, disc_pix_coarse

In [11]:
### Fine pixelation of size approx. beta_t/10 (can be a bit larger than beta_t/10, so using round is fine)
n = round(math.log(np.sqrt(np.pi/3)/(0.1*beta_t), 2)); nside = 2**n; 
vec = hp.pix2vec(nside, hp.ang2pix(nside, disc_center[0], disc_center[1], nest=True, lonlat=True), nest=True)
coarse_scan_pix = hp.ang2pix(nside, coarse_scan_coord[0], coarse_scan_coord[1], nest=True, lonlat=True)            
del vec, coarse_scan_coord

In [12]:
q_pix = np.asarray(hp.ang2pix(nside, data_ra, data_dec, nest=True, lonlat=True)) # healpy pixel number of the stars, needed to find stars near a specific template location

In [13]:
### Compute the template for "step" number of location
print('Starting the template scan...')
sys.stdout.flush()

first_pix = step*i_step; last_pix = min(step*(i_step+1), len(coarse_scan_pix))
tic = tictoc()
fn_template_scan(nside, coarse_scan_pix[first_pix:last_pix])
toc = tictoc()    

print('Template scan completed in', str(toc - tic), 's.')
sys.stdout.flush()

Starting the template scan...
Template scan completed in 225.96506214141846 s.
